# Attempt to fine-tune GPT3 to translate between languages
The first language will be our made up language (see notes on get_bible as to why we think GPT is cheating)
The thesis is that fine-tuning will cause GPT3 to learn the fake language as a new language by assigning the embeddings of the
new words essentially to the same dimensional space as the english.  Should be trivial to translate it back as it is a word 
for word translation.



In [1]:
from lib.config import get_config
import pandas as pd

bible = pd.read_csv('data/bible.bbe.csv')

# Assuming the gospels have a lot of overlap include the synoptic gospels plus John to train
# then predict Mark (which should be easy as Matthew and Luke may have copied from him)
train = bible[bible['book'].isin(['MAT','LUK','JHN'])]
validate = bible[bible['book'].isin(['MRK'])]

In [2]:
train

,0,book,chapter,verse,eng-web,eng-asv,eng-kjv2006,engBBE,engWycliffe,birrig
23213,MAT 1:1,MAT,1,1,"The book of the genealogy of Jesus Christ, the...","The book of the generation of Jesus Christ, th...","The book of the generation of Jesus Christ, th...","The book of the generations of Jesus Christ, t...","The book of the generacioun of Jhesu Crist, th...","Lxi fuuc us lxi girivelourz us Yizaz Kxvozl, l..."
23214,MAT 1:2,MAT,1,2,Abraham became the father of Isaac. Isaac beca...,Abraham begat Isaac; and Isaac begat Jacob; an...,Abraham begat Isaac; and Isaac begat Jacob; an...,The son of Abraham was Isaac; and the son of I...,Abraham bigat Isaac. Isaac bigat Jacob. Jacob ...,Lxi zur us Efvexen hez Ozeek; erw lxi zur us O...
23215,MAT 1:3,MAT,1,3,Judah became the father of Perez and Zerah by ...,and Judah begat Perez and Zerah of Tamar; and ...,And Judas begat Phares and Zara of Thamar; and...,And the sons of Judah were Perez and Zerah by ...,"Judas bigat Fares and Zaram, of Tamar. Fares b...",Erw lxi zurz us Yawex hivi Qivid erw Divex fj ...
23216,MAT 1:4,MAT,1,4,Ram became the father of Amminadab. Amminadab ...,and Ram begat Amminadab; and Amminadab begat N...,And Aram begat Aminadab; and Aminadab begat Na...,And the son of Ram was Amminadab; and the son ...,Esrom bigat Aram. Aram bigat Amynadab. Amynada...,Erw lxi zur us Ven hez Ennorewef; erw lxi zur ...
23217,MAT 1:5,MAT,1,5,Salmon became the father of Boaz by Rahab. Boa...,and Salmon begat Boaz of Rahab; and Boaz begat...,And Salmon begat Booz of Rachab; and Booz bega...,And the son of Salmon by Rahab was Boaz; and t...,"Salmon bigat Booz, of Raab. Booz bigat Obeth, ...",Erw lxi zur us Zepnur fj Vexef hez Fued; erw l...
...,...,...,...,...,...,...,...,...,...,...
26987,JHN 21:21,JHN,21,21,"Peter, seeing him, said to Jesus, “Lord, what ...","Peter therefore seeing him saith to Jesus, Lor...","Peter seeing him saith to Jesus, Lord, and wha...","Seeing him, Peter said to Jesus, What about th...","Therfor whanne Petre hadde seyn this, he seith...","Ziiorg xon, Qiliv zeow lu Yizaz, Hxel efual lx..."
26988,JHN 21:22,JHN,21,22,"Jesus said to him, “If I desire that he stay u...","Jesus saith unto him, If I will that he tarry ...","Jesus saith unto him, If I will that he tarry ...","Jesus said to him, If it is my desire for him ...","Jhesus seith to him, So I wole that he dwelle ...","Yizaz zeow lu xon, Os ol oz nj wizovi suv xon ..."
26989,JHN 21:23,JHN,21,23,This saying therefore went out among the broth...,This saying therefore went forth among the bre...,Then went this saying abroad among the brethre...,So this saying went about among the brothers t...,Therfor this word wente out among the brithere...,Zu lxoz zejorg hirl efual enurg lxi fvulxivz l...
26990,JHN 21:24,JHN,21,24,This is the disciple who testifies about these...,This is the disciple that beareth witness of t...,This is the disciple which testifieth of these...,This is the disciple who gives witness about t...,"This is thilke disciple, that berith witnessyn...",Lxoz oz lxi wozkoqpi hxu gomiz holrizz efual l...


In [64]:
df2 = pd.DataFrame()
for item in ['eng-web', 'eng-asv', 'eng-kjv2006']:
    df2 = df2.append(pd.DataFrame({'prompt': train[item], 'completion': train['birrig']}))

# removal all null rows
df2 = df2.dropna()

df2['completion'] = df2['completion'].apply(lambda x: " " + x + '\n###')
df2['prompt'] = df2['prompt'].apply(lambda x: x + "\n\n###\n\n")
df2.to_json('data/train_exp3.jsonl', orient='records', lines=True)
df2


/var/folders/tq/94d0g6xx05x_h_z4rfdfh6bh0000gn/T/ipykernel_32654/497036072.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(pd.DataFrame({'prompt': train[item], 'completion': train['birrig']}))
/var/folders/tq/94d0g6xx05x_h_z4rfdfh6bh0000gn/T/ipykernel_32654/497036072.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(pd.DataFrame({'prompt': train[item], 'completion': train['birrig']}))
/var/folders/tq/94d0g6xx05x_h_z4rfdfh6bh0000gn/T/ipykernel_32654/497036072.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(pd.DataFrame({'prompt': train[item], 'completion': train['birrig']}))


,prompt,completion
23213,"The book of the genealogy of Jesus Christ, the...","Lxi fuuc us lxi girivelourz us Yizaz Kxvozl, ..."
23214,Abraham became the father of Isaac. Isaac beca...,Lxi zur us Efvexen hez Ozeek; erw lxi zur us ...
23215,Judah became the father of Perez and Zerah by ...,Erw lxi zurz us Yawex hivi Qivid erw Divex fj...
23216,Ram became the father of Amminadab. Amminadab ...,Erw lxi zur us Ven hez Ennorewef; erw lxi zur...
23217,Salmon became the father of Boaz by Rahab. Boa...,Erw lxi zur us Zepnur fj Vexef hez Fued; erw ...
...,...,...
26987,"Peter seeing him saith to Jesus, Lord, and wha...","Ziiorg xon, Qiliv zeow lu Yizaz, Hxel efual l..."
26988,"Jesus saith unto him, If I will that he tarry ...","Yizaz zeow lu xon, Os ol oz nj wizovi suv xon..."
26989,Then went this saying abroad among the brethre...,Zu lxoz zejorg hirl efual enurg lxi fvulxivz ...
26990,This is the disciple which testifieth of these...,Lxoz oz lxi wozkoqpi hxu gomiz holrizz efual ...


In [65]:

# set environment variable in ipython notebook
import os
os.environ["OPENAI_API_KEY"] = get_config('openai')['api_key']




In [66]:
!python3 /Users/chrispriebe/Library/Python/3.8/lib/python/site-packages/openai/_openai_scripts.py tools fine_tunes.prepare_data -f data/train_exp3.jsonl -q

15099.91s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Analyzing...

- Your file contains 9296 prompt-completion pairs
- Based on your data it seems like you're trying to fine-tune a model for classification
- For classification, we recommend you try one of the faster and cheaper models, such as `ada`
- For classification, you can estimate the expected model performance by keeping a held out dataset, which is not used for training
- There are 280 duplicated prompt-completion sets. These are rows: [1395, 3178, 3224, 3427, 3461, 3562, 3750, 3830, 3866, 3964, 3967, 3983, 4012, 4117, 4126, 4176, 4266, 4319, 4323, 4324, 4326, 4331, 4333, 4334, 4335, 4336, 4337, 4461, 4492, 4682, 4767, 5084, 5121, 5283, 5317, 5318, 5329, 5344, 5422, 5458, 5576, 5799, 5811, 5829, 6046, 6195, 6231, 6260, 6263, 6269, 6287, 6288, 6289, 6291, 6292, 6326, 6335, 6343, 6347, 6353, 6367, 6369, 6384, 6385, 6389, 6421, 6433, 6436, 6448, 6474, 6480, 6493, 6497, 6522, 6524, 6538, 6539, 6576, 6587, 6599, 6645, 6647, 6651, 6652, 6664, 6674, 6685, 6706, 6732, 6738, 6756, 6766, 

In [67]:
!python3 /Users/chrispriebe/Library/Python/3.8/lib/python/site-packages/openai/_openai_scripts.py api fine_tunes.create -t "data/train_exp3.jsonl"

15195.11s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Upload progress: 100%|████████████████████| 2.60M/2.60M [00:00<00:00, 2.93Git/s]
Uploaded file from data/train_exp3.jsonl: file-2UiIGZOEQZwEGc1X9sZjdQxm
Created fine-tune: ft-LNBk5IklByRwDXL0ej9fdAks
Streaming events until fine-tuning is complete...

(Ctrl-C will interrupt the stream, but not cancel the fine-tune)
[2023-03-29 17:32:32] Created fine-tune: ft-LNBk5IklByRwDXL0ej9fdAks

Stream interrupted (client disconnected).
To resume the stream, run:

  openai api fine_tunes.follow -i ft-LNBk5IklByRwDXL0ej9fdAks



In [69]:
!python3 /Users/chrispriebe/Library/Python/3.8/lib/python/site-packages/openai/_openai_scripts.py api fine_tunes.follow -i ft-LNBk5IklByRwDXL0ej9fdAks

26134.01s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


[2023-03-29 17:32:32] Created fine-tune: ft-LNBk5IklByRwDXL0ej9fdAks
[2023-03-29 17:34:34] Fine-tune costs $10.46
[2023-03-29 17:34:35] Fine-tune enqueued. Queue number: 0
[2023-03-29 17:45:02] Fine-tune started
[2023-03-29 17:50:13] Completed epoch 1/4
[2023-03-29 17:54:21] Completed epoch 2/4
[2023-03-29 18:02:37] Completed epoch 4/4
[2023-03-29 18:02:54] Uploaded model: curie:ft-personal-2023-03-30-01-02-54
[2023-03-29 18:02:55] Uploaded result file: file-buUj6uUAdoQb6BB8cyr16W8W
[2023-03-29 18:02:55] Fine-tune succeeded

Job complete! Status: succeeded 🎉
Try out your fine-tuned model:

openai api completions.create -m curie:ft-personal-2023-03-30-01-02-54 -p <YOUR_PROMPT>


In [70]:
import openai
from lib.cipher import substitution_cipher
from nltk.translate.bleu_score import sentence_bleu

openai.api_key = os.getenv("OPENAI_API_KEY")

def translate(text):
    """
    Translate text from English to Birrig
    
    Parameters
    ----------
    text: str|array
        The text to translate. If an array is passed, each element will be translated.

    Returns
    ------- 
    array
        An array of translations
        
    """
    if isinstance(text, list):
        text = [row + "\n\n###\n\n" for row in text]
        if len(text) > 20:
            # break it into chunks of 20 and call translate on each chunk
            # then combine the results
            result = []
            for i in range(0, len(text), 20):
                result += translate(text[i:i+20])
            return result

    else:
        text += "\n\n###\n\n"

    response = openai.Completion.create(
        model="curie:ft-personal-2023-03-30-01-02-54",
        prompt=text,
        temperature=0.5,  # 0.1 is very little randomness/creativity, 2 is very
        max_tokens=100,
        n=1,
        frequency_penalty=0,
        presence_penalty=0,
        stop=["\n###"]
    )
    result = []
    for choice in response.get('choices',[{}]):
        decoded = substitution_cipher(choice.get('text').strip(), encode=False).strip()
        result.append(decoded)
        # result.append({
        #     'translation': choice.get('text').strip(),
        #     'decoded_translation': decoded,
        #     'bleu_score': sentence_bleu([text.split()], decoded.split())
        # })
    
    return result

In [73]:
# Get the column eng-web from the first 3 rows of validate then send it to translate
validate['translation'] = translate(validate['eng-web'].to_list())
validate

/var/folders/tq/94d0g6xx05x_h_z4rfdfh6bh0000gn/T/ipykernel_32654/591098392.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  validate['translation'] = translate(validate['eng-web'].to_list())


,0,book,chapter,verse,eng-web,eng-asv,eng-kjv2006,engBBE,engWycliffe,birrig,translation,bleu_score
24284,MRK 1:1,MRK,1,1,The beginning of the Good News of Jesus Christ...,"The beginning of the gospel of Jesus Christ, t...","The beginning of the gospel of Jesus Christ, t...",The first words of the good news of Jesus Chri...,"The bigynnyng of the gospel of Jhesu Crist, th...",Lxi sovzl huvwz us lxi guuw rihz us Yizaz Kxvo...,"The first of the good news of Jesus Christ, th...",0.880112
24285,MRK 1:2,MRK,1,2,"As it is written in the prophets, “Behold, I s...","Even as it is written in Isaiah the prophet, B...","As it is written in the prophets, Behold, I se...",Even as it is said in the book of Isaiah the p...,"As it is writun in Ysaie, the prophete, Lo! Y ...",Imir ez ol oz zeow or lxi fuuc us Ozeoex lxi q...,"As it says in the prophets, See, I send my ser...",0.708352
24286,MRK 1:3,MRK,1,3,"the voice of one crying in the wilderness, ‘Ma...","The voice of one crying in the wilderness, Mak...","The voice of one crying in the wilderness, Pre...","The voice of one crying in the waste land, Mak...","The vois of a crier in desert, Make ye redi th...","Lxi muoki us uri kvjorg or lxi hezli perw, Nec...","The voice of one crying in the waste land, Mak...",0.735874
24287,MRK 1:4,MRK,1,4,John came baptizing in the wilderness and prea...,"John came, who baptized in the wilderness and ...","John did baptize in the wilderness, and preach...","John came, and gave baptism in the waste land,...","Joon was in desert baptisynge, and prechynge t...","Yuxr keni, erw gemi feqlozn or lxi hezli perw,...",And John came giving baptism in the waste land...,0.115310
24288,MRK 1:5,MRK,1,5,All the country of Judea and all those of Jeru...,And there went out unto him all the country of...,And there went out unto him all the land of Ju...,And there went out to him all the people of Ju...,"And al the cuntre of Judee wente out to hym, a...",Erw lxivi hirl ual lu xon epp lxi qiuqpi us Ya...,And all the country of Judaea and all those of...,0.523172
...,...,...,...,...,...,...,...,...,...,...,...,...
24957,MRK 16:16,MRK,16,16,He who believes and is baptized will be saved;...,He that believeth and is baptized shall be sav...,He that believeth and is baptized shall be sav...,He who has faith and is given baptism will get...,"Who that bileueth, and is baptisid, schal be s...",Xi hxu xez seolx erw oz gomir feqlozn hopp gil...,He who has faith and gives baptism will be sal...,0.218497
24958,MRK 16:17,MRK,16,17,These signs will accompany those who believe: ...,And these signs shall accompany them that beli...,And these signs shall follow them that believe...,And these signs will be with those who have fa...,"And these tokenes schulen sue hem, that bileue...",Erw lxizi zogrz hopp fi holx lxuzi hxu xemi se...,And these signs will come with those who have ...,0.664994
24959,MRK 16:18,MRK,16,18,they will take up serpents; and if they drink ...,"they shall take up serpents, and if they drink...",They shall take up serpents; and if they drink...,"They will take up snakes, and if there is pois...",thei schulen do awei serpentis; and if thei dr...,"Lxij hopp leci aq zreciz, erw os lxivi oz quoz...",They will take life and if any disease will be...,0.191983
24960,MRK 16:19,MRK,16,19,"So then the Lord, after he had spoken to them,...","So then the Lord Jesus, after he had spoken un...","¶ So then after the Lord had spoken unto them,...","So then the Lord Jesus, after he had said thes...","And the Lord Jhesu, aftir he hadde spokun to h...","Zu lxir lxi Puvw Yizaz, esliv xi xew zeow lxiz...","Then, when he had said to them, the Lord was t...",0.466981


In [74]:
# drop rows that have a null value in any column
validate = validate.dropna()
validate['bleu_score'] = validate.apply(lambda row: sentence_bleu([row['eng-web'].split(), row['eng-asv'].split(), row['eng-kjv2006'].split(),row['engBBE'].split()], row['translation'].split()), axis=1)

/Users/chrispriebe/Library/Python/3.8/lib/python/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/Users/chrispriebe/Library/Python/3.8/lib/python/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/Users/chrispriebe/Library/Python/3.8/lib/python/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider 

In [75]:
# describe the bleu score column
validate['bleu_score'].describe()

count     6.730000e+02
mean      3.856940e-01
std       2.568575e-01
min      1.234056e-231
25%       2.204887e-01
50%       3.828752e-01
75%       5.531961e-01
max       1.000000e+00
Name: bleu_score, dtype: float64

## Analysis
This theory proposes that to avoid word for word translation start with the web english version and target the BBE english version.  That way it has to use alternate words to find the way.

It generally does okay and makes mostly readable text.  Some weaknesses observed are
 - might get distracted and start repeating itself
 - likes to start most verses with "And"

| Stat | Number
| ----- | ----- |
| Average bleu score | 0.386 |
| 75 percentile | 0.532 |

In [72]:
translate("For God so loved the world that he gave his only Son, so that everyone who believes in him may not die but have eternal life.")

['For God had such love for the world that he gave his only son, so that a man who has faith in him may not come to death but have eternal life.']

In [76]:
translate("he said let there be light and there was light")



['He said, Let the light be then, and the light came:']

In [77]:
# Try some unknown words
translate("Bongo bongo I love you, gone to Venus with a hole in my shoe")

['Go on bed, I have love for you, and underto the seed of Ramon to have a hand in my shoes:']

In [78]:
translate ("Ship Pit! Pirate ahoy-lay!")

['The strength! The servants!']

In [79]:
translate("Sheep went baa")

['And keeping away,']

In [80]:
translate("Boat a brother on a mountain")

['To be hand on a brother on a mountain;']

In [84]:
translate("Be kind and play with rocks!")

['Be certain, do their stones and take their plaeses streed.']

In [35]:
translate("SOS! Ship overboard!! Lost ninty percent of people!")

[{'translation': 'Ziiw ziir, ziiw ziir, ziiw ziir, ual uvwivz!',
  'decoded_translation': 'Seed seen, seed seen, seed seen, out orders!',
  'bleu_score': 0}]